# Confronto Insertion-sort e heap sort
> Laboratorio di Algoritmi - Sergio Cibecchini - 7049795

In questo notebook verranno analizzati e confrontati i due algoritmi di ordinamento *insertion-sort* e *heap-sort*. 

## Studio prestazioni attese
Prima di procedere con l'implementazione degli algoritmi è opportuno fare delle assunzioni sulle prestazioni che ci aspettiamo.

## Insertion sort

L'insertion sort è un algoritmo di ordinamento **stabile** che ordina **sul posto** ossia opera sull'array da ordinare senza aver bisogno di memoria aggiuntiva. 

La logica secondo cui agisce questo algoritmo si potrebbe paragonare al modo in cui viene solitamente ordinato un mazzo di carte: l'insertion sort opera infatti ordinando gradualmente un sottoarray sinistro dell'input fornito, incrementando di 1 la grandezza del sottoarray ad ogni iterazione fino a quando l'intero array non è ordinato.
#### Costo Insertion sort

Possiamo quindi dedurre che nel **caso migliore**, ossia quando l'input è un array ordinato, l'insertion sort analizzera ogni elemento dell'array fino alla fine senza operare alcun cambiamento e dunque avremo un **costo lineare $O(n)$**. 

Nel **caso peggiore** ossia con in input un array ordinato al contrario l'array avrà un **costo quadratico $O(n^2)$**. 

Anche nel **caso medio** il costo sarà **quadratico $O(n^2)$**.
#### Esempio applicazione insertion sort

<img src = "insertion.png" alt = "esempio insertion sort" width = 50%/>

## Heap sort

L'heap sort è un algoritmo di ordinamento tipicamente **non stabile** basato su confronti che ordina **sul posto**. 

L'heap sort opera trasformando un array in un max-heap, un albero binario rappresentato utilizzando un array dove il massimo valore si trova alla radice e scendendo dalla radice verso le foglie i valori diminuiscono, ossia vale la proprietà: 
$$A[parent(i)] \ge A[i]$$

La rappresentazione tramite array di un albero binario sfrutta le seguenti proprietà: 
- Il primo elemento dell'array è la radice dell'albero
- Il padre del nodo $i$ è il nodo $\lfloor i/2 \rfloor$
- Il figlio sinistro del nodo  $i$ è il nodo $2i$
- Il figlio destro del nodo  $i$ è il nodo $2i + 1$
#### Esempio rapresentazione di un albero binario tramite array

<img src = "Array-Complete-Tree.png" alt = "esempio insertion sort" width = 50%/>

#### Costo Insertion sort

Lo heap sort è composto da due parti, *Max-Heapify* e *Build-Max-Heap*. Max-Heapify ha costo $O(lgn)$, mentre Build-max-heap effettua $n/2$ chiamate a Max-heapify, dunque abbiamo un costo complessivo di $O(n lgn)$.

## Test effettuati
Gli algoritmi saranno testati secondo tre modalità: 
- Con valori randomizzati
- Con valori crescenti 
- Con valori decrescenti

I risultati dei test saranno rappresentati tramite la libreria *matplotlib*. 

## Codice implementato

#### Import delle librerie necessarie

> In caso si verifichi un *ModuleNotFoundError* per matplotlib, cambiare False a True ed eseguire la riga sottostante: 

In [1]:
if False: 
    import sys  
    !{sys.executable} -m pip install --user matplotlib

#### Librerie utilizzate: 

In [2]:
import random
from timeit import default_timer as timer
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

## Codice insertion sort

In [3]:
def insertionSort(A):
    for j in range(1, len(A)): #partiamo dal secondo elemento 
        key = A[j] #impostiamo l'elemento con cui confrontare
        i = j - 1 
        while i >= 0 and A[i] > key: 
            A[i + 1] = A[i]
            i -= 1
        A[i + 1] = key # lo slot corretto per key è i + 1

## Codice heapSort

In [4]:
def maxHeapify(arr, heapSize, i):
    l = left(i)  # imposta l'indice del figlio sinistro
    r = right(i)  # imposta l'indice del figlio destro

    if l < heapSize and arr[i] < arr[l]:  # controlliamo di non aver sforato l'array e confrontiamo i e left
        maxVal = l
    else:
        maxVal = i

    if r < heapSize and arr[r] > arr[maxVal]:
        maxVal = r
    # se i non è il valore più grande allora dobbiamo scambiarlo con il maggiore per mantenere la proprietà
    # di max heap
    if maxVal != i:
        arr[i], arr[maxVal] = arr[maxVal], arr[i]  # scambia a[i] con a[max]
        maxHeapify(arr, heapSize, maxVal)


In [5]:
def heapSort(arr):
    buildMaxHeap(arr) #costruisce un max-heap
    fixPositions(arr, len(arr)) #aggiusta i valori in modo che siano crescenti

In [6]:
def fixPositions(arr, heapSize):
    for i in range(heapSize - 1, 0, -1):  # scorre l'array al contrario
        arr[i], arr[0] = arr[0], arr[i]  # scambia zero con il valore massimo del sottoarray che va da zero ad i
        # stiamo chiamando maxheapify su un array di grandezza i, stiamo ponendo in posizione zero il valore
        # più grande del sottoarray
        maxHeapify(arr, i, 0)


def buildMaxHeap(arr):
    heapSize = len(arr) 
    for i in range(heapSize // 2 - 1, -1, -1): 
        maxHeapify(arr, heapSize, i)


def left(i): #ritorna l'elemento sinistro nella rappresentazione di alberi binari tramite array
    return 2 * i + 1


def right(i): #ritorna l'elemento destro nella rappresentazione di alberi binari tramite array
    return 2 * i + 2

## Classe Tester

La classe tester esegue i test secondo le tre modalità descritte in precedenza e restituisce il risultato.

In [7]:
class Tester:
    def __init__(self, n, testedAlgorithms):  # forniamo in ingresso il valore massimo del vettore di test
        self.incrementalArrays = None
        self.randomArrays = None
        self.n = n
        self.testedAlgorithms = testedAlgorithms  # numero di algoritmi che stiamo testando

    def generateIncrementalValues(self):  # genera array con valori sequenziali da 0 a n -1
        self.incrementalArrays = []
        self.initializeMultidimensionalArray(self.incrementalArrays, self.n)
        self.incrementalArrays[0] = [0]
        for i in range(1, self.n):
            self.incrementalArrays[i] = self.incrementalArrays[i - 1].copy()
            self.incrementalArrays[i].append(i)

    def generateRandomValues(self, lowerB, upperB):  # genera array con valori randomici
        self.randomArrays = []
        self.initializeMultidimensionalArray(self.randomArrays, self.n)
        for i in range(0, self.n):
            for j in range(0, i + 1):
                self.randomArrays[i].append(random.randrange(lowerB, upperB))

    def initializeMultidimensionalArray(self, array, end):  # converte l'array in input in un array di array
        for i in range(0, end):
            array.append([])

    def runRandomizedTests(self, lowerB, upperB):  # chiama runtTests con in input gli array randomici
        self.generateRandomValues(lowerB, upperB)
        return self.runTest(self.randomArrays)

    def runIncrementalTests(self):  # chiama runtTests con in input gli array con valori crescenti
        self.generateIncrementalValues()
        return self.runTest(self.incrementalArrays)

    def runDecrementalTests(self):  # inverte gli array con valori crescenti in modo che siano decrescenti
        self.generateIncrementalValues()
        data = []
        for i in range(0, len(self.incrementalArrays)):
            data.append(list(reversed(self.incrementalArrays[i])))
        return self.runTest(data)

    def runTest(self, array):  # metodo principale per il testing, effettua i test con gli array in input
        results = []
        self.initializeMultidimensionalArray(results, self.testedAlgorithms)
        data = []
        for i in range(0, self.testedAlgorithms):  # siccome gli algoritmi operano in place è necessario
            data.append(array.copy())  # creare due copie dello stesso input in modo che ad uno non siano forniti
            # gli array già ordinati

        for j in range(0, self.n):
            index = 0
            previous = results[index][j - 1][1] if j != 0 else None  # imposta il valore del precedente
            results[index].append((j, self._testInsertion(data[index][j], previous)))  # testa l'insertion sort
            index += 1
            previous = results[index][j - 1][1] if j != 0 else None
            results[index].append((j, self._testHeapSort(data[index][j], previous)))  # testa l'heap sort
        return results

    def _testInsertion(self, data, previous):  # fornisce all'algoritmo l'array da ordinare
        start = timer()
        insertionSort(data)
        end = timer()
        # restituisce il tempo impiegato per l'ordinamento
        return end - start if previous is None else end - start + previous

    def _testHeapSort(self, data, previous):
        start = timer()
        heapSort(data)
        end = timer()
        return end - start if previous is None else end - start + previous


## Classe Plotter

La classe plotter permette di realizzare i plot dei risultati forniti in input

In [8]:
class Plotter:
    #semplifica il plotting del grafico
    def plotGraph(self, x, toPlot, graphLabels, colors, graphTitle, axisLabels):
        for i in range(0, len(toPlot)):
            plt.plot(x, toPlot[i], colors[i])
        plt.legend(graphLabels, title=graphTitle)
        plt.xlabel = axisLabels[0]
        plt.ylabel = (axisLabels[1])
        plt.show()


Il metodo *getYAxis* converte i risultati forniti da Tester in un array interpretabile dalla classe Plotter. 

In [9]:
def getYAxis (result, index): #restituisce soltato l'asse 
    y = []
    for val in result[index]:
        y.append(val[1])
    return y

## Dati di input i test
Nelle righe di codice sottostanti sono presenti i dati che vengono forniti in ingresso alla classe tester per il run dei test.  
#### Parametro n
Il parametro n rappresenta la grandezza massima degli array analizzati, ad esempio con n uguale a 10 verranno analizzati 10 array, il primo con grandezza 1 e l'ultimo con grandezza 10. 

In [10]:
n = 500 #grandezza massima array

#### Lower e Upper Bound
Lower e Upper bound rappresentano i parametri che saranno utilizzati nella generazione dei valori randomici tramite la funzione *random.randrange*. In particolare andremo a generare dei valori da inserire nell'array nel range $[lowerB, upperB)$, dunque: 
$$ \forall i,j \ randomArrays[i][j] \in [lowerB, upperB - 1]$$

In [11]:
lowerB = 0
upperB = 100 

#### Numtested 
Rappresenta il numero di algoritmi che stiamo testando, in questo caso vale 2. 

In [12]:
numTested = 2 #don't change this value

## Run dei test
Con le righe di codice successive andremo a creare un oggetto della classe *Tester* che utilizzeremo per effettuare il run dei test. L'output verrà memorizzato e rappresentato successivamente tramite grafici. 

#### Creazione istanza Tester

In [13]:
t = Tester(n, numTested) #creiamo una istanza di Tester con i parametri modificati sopra

#### Run del test con array a valori randomizzati 

In [14]:
randomizedTestResult = t.runRandomizedTests(lowerB,upperB)

#### Run del test con array a valori crescenti

In [15]:
incrementalTestResult = t.runIncrementalTests()

#### Run del test a valori decrescenti

In [16]:
decrementalTestResult = t.runDecrementalTests()

## Plotting e analisi dei risultati
Per effettuare il plotting andremo a creare una istanza della classe *Plotter* e gli forniremo i risultati ottenuti alla sezione di *Run dei test*. 
Per semplificare il run andremo a memorizzare tutti i risultati in un array y, dove: 
- y\[0\] risultati **insertionSort** con array **randomizzato**
- y\[1\] risultati **heapSort** con array **randomizzato**
- y\[2\] risultati **insertionSort** con array a valori **crescenti**
- y\[3\] risultati **heapSort** con array a valori **crescenti**
- y\[4\] risultati **insertionSort** con array a valori **decrescenti** 
- y\[5\] risultati **heapSort** con array a valori **decrescenti**



In [17]:
y = [] #asse y
r = [randomizedTestResult, incrementalTestResult, decrementalTestResult]
for i in range(0, len(r)):
    for j in range(0, len(r[i])):
        y.append(getYAxis(r[i], j))
x = [i for i in range(0, n)] #asse x

#### Creazione istanza di classe Plotter
Creiamo una istanza della classe Plotter per poter rappresentare i grafici. 

In [18]:
p = Plotter()

## Studio casi InsertionSort
Per prima cosa analizziamo l'insertionSort nei tre casi. Gli input sono stati scelti appositamente in modo da poter verificare caso peggiore, medio e migliore dell'insertionSort. 

In [19]:
p.plotGraph(x, [y[0], y[2], y[4]], ["randomized", "increasing", "decreasing"], 
            ["-r", "-m", "-g"], "InsertionSort con i diversi input", ["nNodi", "tempo"])

Come possiamo osservare dal grafico, gli input generano effettivamente caso peggiore, medio e migliore. 
Vediamo infatti che il caso di array con valori randomizzati segue una curva $O(n^2)$ ma con andamento meno ripido rispetto al caso con array a valori decrescenti. Il caso con array già ordinati invece segue un andamento lineare.

Abbiamo quindi potuto osservare nella pratica i tre casi del costo dell'insertionSort, confermando nella pratica le ipotesi teoriche. 

## Studio casi HeapSort
Analizziamo ora l'HeapSort nelle tre casistiche: 

In [20]:
p.plotGraph(x, [y[1], y[3], y[5]], ["randomized", "increasing", "decreasing"], 
            ["-r", "-m", "-g"], "HeapSort con i diversi input", ["nNodi", "tempo"])

Dal grafo possiamo dedurre che l'Heapsort non risente in modo rilevante dall'ordinamento dei valori, in quanto otteniamo una performance simile per tutti e tre i casi di esame. 

## Confronto InsertionSort e HeapSort
Analizziamo i due algoritmi nelle varie casistiche. 
### Caso valori randomici

In [21]:
p.plotGraph(x, [y[0], y[1]], ["insertion random values", "heap random values"], 
            ["-r", "-g"], "Confronto insertionSort e heapSort con valori randomici", ["nNodi", "tempo"])

Possiamo osservare che nel caso di valori randomici la differenza di andamento tra i due algoritmi, $O(n^2)$ per l'insertion e $O(n lg n)$ per l'heap, viene rispecchiata dal grafico

### Caso valori crescenti

In [22]:
p.plotGraph(x, [y[2], y[3]], ["insertion random values", "heap random values"], 
            ["-r", "-g"], "Confronto insertionSort e heapSort con valori crescenti", ["nNodi", "tempo"])

Nel caso migliore del insertion invece, vediamo che l'heap sort si rivela più lento, in quanto stiamo confrontando un $O(n)$ con $O(n lg n)$. 

### Caso valori decrescenti

In [23]:
p.plotGraph(x, [y[4], y[5]], ["insertion random values", "heap random values"], 
            ["-r", "-g"], "Confronto insertionSort e heapSort con valori decrescenti", ["nNodi", "tempo"])

Nel caso degli array ordinati al contrario la differenza tra insertion e heap è ancora più accentuata di quanto visto nel caso con valori randomici.

## Conclusioni
Tramite questo esperimento è stato possibile mettere a confronto l'andamento degli algoritmi *insertionSort* e *heapSort* al variare della tipologia di input. 
Le rilevazioni sperimentali hanno confermato quanto già supposto a priori nella teoria, mostrando che, sebbene i due algoritmi abbiano la stessa complessità spaziale, l'heapSort è più efficace in tutti i casi tranne nel caso migliore dell'insertion sort. 

L'heapSort si conferma dunque la scelta migliore tra i due nel caso si vogliano ordinare un numero consistente di elementi e la nostra priorità sia minimizzare il tempo di esecuzione. 